In [1]:
import json
import pandas as pd
import numpy as np

mc_data_path = "/content/drive/MyDrive/DATA 301/DATA301 Project/mattcuento data"
mc_prefix = "mattcuento_"

ModuleNotFoundError: No module named 'pandas'

In [20]:
identifiers_string = open(f"{mc_data_path}/{mc_prefix}Identifiers.json", "r").read()
identifiers_json = json.loads(identifiers_string)

inferences_string = open(f"{mc_data_path}/{mc_prefix}Inferences.json", "r").read()
inferences_json = json.loads(inferences_string)

marquee_string = open(f"{mc_data_path}/{mc_prefix}Marquee.json", "r").read()
marquee_json = json.loads(marquee_string)

playlist_string = open(f"{mc_data_path}/{mc_prefix}Playlist1.json", "r").read()
playlist_json = json.loads(playlist_string)

search_string = open(f"{mc_data_path}/{mc_prefix}SearchQueries.json", "r").read()
search_json = json.loads(search_string)

stream_0_string = open(f"{mc_data_path}/{mc_prefix}StreamingHistory0.json", "r").read()
stream_json = json.loads(stream_0_string)

stream_1_string = open(f"{mc_data_path}/{mc_prefix}StreamingHistory1.json", "r").read()
stream_json.extend(json.loads(stream_1_string))

voice_string = open(f"{mc_data_path}/{mc_prefix}VoiceInput.json", "r").read()
voice_json = json.loads(voice_string)

library_string = open(f"{mc_data_path}/{mc_prefix}YourLibrary.json", "r").read()
library_json = json.loads(library_string)

In [21]:
print(stream_json)

[{'endTime': '2020-05-18 21:02', 'artistName': "Maddy O'Neal", 'trackName': 'Like Gold - Birocratic Remix', 'msPlayed': 6857}, {'endTime': '2020-11-06 00:00', 'artistName': 'David Bowie', 'trackName': "Let's Dance - 2002 Remaster", 'msPlayed': 247600}, {'endTime': '2020-11-06 00:04', 'artistName': 'David Bowie', 'trackName': 'Ashes to Ashes - 2017 Remaster', 'msPlayed': 266160}, {'endTime': '2020-11-06 00:07', 'artistName': 'David Bowie', 'trackName': 'Changes - 2015 Remaster', 'msPlayed': 217746}, {'endTime': '2020-11-06 00:10', 'artistName': 'David Bowie', 'trackName': 'Sound and Vision - 2017 Remaster', 'msPlayed': 184381}, {'endTime': '2020-11-06 00:13', 'artistName': 'David Bowie', 'trackName': 'Repetition - ChangesNowBowie Version', 'msPlayed': 181840}, {'endTime': '2020-11-06 00:17', 'artistName': 'David Bowie', 'trackName': 'Modern Love - Single Version; 1990 Remaster', 'msPlayed': 237666}, {'endTime': '2020-11-06 00:20', 'artistName': 'David Bowie', 'trackName': 'Ziggy Stardus